In [3]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jul 19 16:44:55 2019

@author: Dylan
"""

#import all modules
import pandas as pd
import sys
from time import time

### for ROC curve metrics
from sklearn import metrics

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dask import delayed

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from skimage.data import lfw_subset
from skimage.transform import integral_image
from skimage.feature import haar_like_feature
from skimage.feature import haar_like_feature_coord
from skimage.feature import draw_haar_like_feature

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# import Functions

# working directories
# --- change file path for you computer
# jeremy
# marijn
main = "/home/dylan/Projects/Tetse_proj/Dylan/"
data_2 = "/home/dylan/Projects/Tetse_proj/Dylan/2/"


# Create training data set
## 1) create training dataset of labeled coordinates for all sampled images

In [56]:

# Read in labeled data from text file and create indexable dataframe

df = pd.read_fwf(data_2 + 'refined_data.txt',header=None)
which = lambda lst:list(np.where(lst)[0])
new = df.iloc[:,4].str.split(" ", n = 23, expand = True)
df= df.drop(labels=4, axis=1)
result = pd.concat([df, new], axis=1, sort=False)
        # column numbers were messed up, make them go according to range
result.columns = range(result.shape[1])
result.reset_index(inplace=True, drop=True)

text_data = result.copy() 


In [57]:

# Create pixel sampling function for a single image

def pix_sample(image, imgname, i):

    coord_label_df = pd.DataFrame(columns=['label', 'coord', 'Image_name'])
    x1landmark = float(result.loc[i,15])
    y1landmark = float(result.loc[i,16])

    R= 9 ### with R 9 the second plot is quite hard to tell whether it works 
    Rmax= 300

    npospix = 100
    nnegpix = 2*npospix

    a = np.random.uniform(0,1,npospix) * 2 * np.pi
    r = R * np.sqrt(np.random.uniform(0,1,npospix))

    ## If you need it in Cartesian coordinates
    xwithin = r * np.cos(a)
    ywithin = r * np.sin(a)

    outer_radius =Rmax*Rmax
    inner_radius = R*R
    rho= np.sqrt(np.random.uniform(inner_radius, 
                             outer_radius, size=nnegpix))

    theta=  np.random.uniform( 0, 2*np.pi, nnegpix)
    xhoop = rho * np.cos(theta)
    yhoop = rho * np.sin(theta)

    for i in range(len(xwithin)):
        df2_within = pd.DataFrame({'label': ['pos'], 'coord': [(x1landmark + xwithin[i],y1landmark + ywithin[i])], 'Image_name':[imgname]})
        coord_label_df = coord_label_df.append(df2_within)
        #print(coord_label_df)
    for i in range(len(xhoop)):
        df2_hoop = pd.DataFrame({'label': ['neg'], 'coord': [(x1landmark + xhoop[i], y1landmark + yhoop[i])], 'Image_name': [imgname]})
        coord_label_df = coord_label_df.append(df2_hoop)
    return(coord_label_df)


In [58]:

# Create a function that generates a dataset of labaled coordinates
# using the pixel sampling function

def create_Tset(text_data):
    df = pd.DataFrame(columns=['label', 'coord', 'Image_name'])
    for i in range(text_data.shape[0]):
        imgname=" ".join(result.loc[i,[0,1,2]])
#    file, ext = os.path.splitext(imgname)
        img = mpimg.imread(data_2 + imgname)
        df = df.append(pix_sample(img , imgname , i ))
    return(df)